In [1]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline

In [2]:
import os
import cv2
from tqdm import tqdm

#### Prepare csv file formats

In [3]:
annotations = pd.DataFrame() # empty for now, will fill later
classes = pd.DataFrame([['nuclei', 0]]) # do not include background
annotations = pd.DataFrame([])
annotations_with_masks = pd.DataFrame([])

#### Fill out annotations

In [4]:
data_base = '../data/stage1_train'
server_data_path = '/storage/astrakhantsev/kaggle-2018-data-science-bowl/data/train/'

In [5]:
counter_all = 0
counter_bad = 0

for img_id in tqdm(os.listdir(data_base)):
    images_folder = '{}/{}/images/'.format(data_base, img_id)
    masks_folder = '{}/{}/masks/'.format(data_base, img_id)
    img_path = '{}/{}'.format(images_folder, os.listdir(images_folder)[0])
    img_server_path = '{}{}/images/{}'.format(server_data_path, img_id, os.listdir(images_folder)[0])

    for mask_id in os.listdir(masks_folder):
        counter_all += 1
        mask_path = '{}/{}'.format(masks_folder, mask_id)
#         mask_img = cv2.imread(mask_path)
        mask = cv2.imread(mask_path, 0)

        _, contours, _ = cv2.findContours(
            mask,
            cv2.RETR_TREE,
            cv2.CHAIN_APPROX_SIMPLE
        )
        
        best_pair = (0, 0, 0, 0)
        best_w = 0
        best_h = 0

        for cnt_num, c in enumerate(contours):
            cnt = contours[cnt_num]
            x, y, w, h = cv2.boundingRect(cnt)

            if w > best_w and h > best_h:
                best_w = w
                best_h = h
                best_pair = (x, y, x + w, y + h)

#             cv2.rectangle(mask_img, (x, y), (x + w, y + h), (255, 0, 0), 2)

        annotations = pd.concat([
            annotations,
            pd.DataFrame([[img_server_path, best_pair[0], best_pair[1], best_pair[2], best_pair[3], 'nuclei']])
        ])
    
        annotations_with_masks = pd.concat([
            annotations_with_masks,
            pd.DataFrame([[img_id, mask_id, best_pair[0], best_pair[1], best_pair[2], best_pair[3], 'nuclei']])
        ])

#         if len(contours) > 1:
#             plt.imshow(mask_img)
#             plt.show()
#             counter_bad += 1

100%|██████████| 664/664 [04:45<00:00,  2.32it/s]


In [6]:
classes

,0,1
0,nuclei,0


In [7]:
annotations.head()

,0,1,2,3,4,5
0,/storage/astrakhantsev/kaggle-2018-data-scienc...,46,180,64,200,nuclei
0,/storage/astrakhantsev/kaggle-2018-data-scienc...,26,247,42,256,nuclei
0,/storage/astrakhantsev/kaggle-2018-data-scienc...,141,164,164,184,nuclei
0,/storage/astrakhantsev/kaggle-2018-data-scienc...,77,204,93,217,nuclei
0,/storage/astrakhantsev/kaggle-2018-data-scienc...,72,234,92,248,nuclei


In [8]:
annotations_with_masks.head()

,0,1,2,3,4,5,6
0,00071198d059ba7f5914a526d124d28e6d010c92466da2...,07a9bf1d7594af2763c86e93f05d22c4d5181353c6d3ab...,46,180,64,200,nuclei
0,00071198d059ba7f5914a526d124d28e6d010c92466da2...,0e548d0af63ab451616f082eb56bde13eb71f73dfda92a...,26,247,42,256,nuclei
0,00071198d059ba7f5914a526d124d28e6d010c92466da2...,0ea1f9e30124e4aef1407af239ff42fd6f5753c09b4c5c...,141,164,164,184,nuclei
0,00071198d059ba7f5914a526d124d28e6d010c92466da2...,0f5a3252d05ecdf453bdd5e6ad5322c454d8ec2d13ef0f...,77,204,93,217,nuclei
0,00071198d059ba7f5914a526d124d28e6d010c92466da2...,2c47735510ef91a11fde42b317829cee5fc04d05a797b9...,72,234,92,248,nuclei


In [9]:
classes.to_csv('../data/train_classes.csv', index=False)
annotations.to_csv('../data/train_annotations.csv', index=False)
annotations_with_masks.to_csv('../data/train_annotations_with_masks.csv', index=False)